# Template Notebook for vrGazeCore-py
This notebook will layout how to run vrGazeCore in a Jupyter notebook. A brief overview:

In [ ]:
# load conda environment TODO: figure out how to do this in notebook

In [ ]:
# import necessary packages
%load_ext autoreload  # allows for autoreloading of scripts if made changes to scripts
%autoreload 2

from vrgaze.parser import get_args_parser, set_paths
from vrgaze.vrgaze import vrGazeCore, vrGazeData

import numpy as np

import glob
from matplotlib import pyplot as plt
import cv2
import numpy as np
import os, sys

import time

In [ ]:
# set main params & paths

base_dir = '/Users/deepasriprasad/Documents/pyGazeCore/'
exclude_scenes = ['practice1','practice2']
pretrial_list = ['fixate', 'sanityTarget']

args = [
    f'--project_dir={base_dir}',
    f'--raw_data_folder=rawData',
    f'--stim_folder=stimuli/combined/',
    f'--plot_fixations',
    f'--plot_density_maps',
    f'--heatmap_timesteps=1',
    f'--make_density_map_gif'
]

parser = get_args_parser()

params = parser.parse_args(args)

for item in pretrial_list:
    params.pretrial_list.append(item)
    
for scene in exclude_scenes:
    params.exclude_scenes.append(scene)
    
paths = set_paths(params)

vrGaze = vrGazeCore(params, paths)

# Batch Processing Subjects

In [ ]:
# list heatmap timesteps

# list subject filenames
subject_fn_list = ['furrow380_V1.txt', 'furrow381_V1.txt']



In [ ]:
# loop through individual subjects
for subject_fn in subject_fn_list:
    
    subject = os.path.splitext(subject_fn)[0]
    
    print(f'Running {subject}')
    
    raw_data = vrGaze.loadRawData(subject_fn)
    raw_data = vrGaze.processRawData(raw_data)
    parsed_data = vrGaze.parseTrials(data=raw_data, subject=subject)
    parsed_data_key = vrGaze.parsedDataKey(parsed_data)
    
    print(f'Running {len(parsed_data)} trials')
    
    start_time = time.time()
    
    for i, trial in enumerate(parsed_data):
        # if manually excluded, skip trial
        if any([scene in trial.trial_name for scene in vrGaze.params.exclude_scenes]):
            print (f"Manually excluded {trial.trial_name} (Trial Number {i}). Skipping!")
            continue
        
        print(f"Running {trial.trial_name} (Trial Index {i})")  
        
        trial = vrGaze.runFindFixations(trial)
        trial = vrGaze.runHeatmapping(trial)
        
        print (time.time() - start_time)